<a href="https://colab.research.google.com/github/susmitmishra125/QA_NLP/blob/main/NLP_Q_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/susmitmishra125/QA_NLP

fatal: destination path 'QA_NLP' already exists and is not an empty directory.


In [4]:
!pip install contractions

In [5]:
# Importing modules
import nltk
nltk.download('punkt') # For tokenizers
from nltk.tokenize import word_tokenize,sent_tokenize
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
import contractions
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
# importing dataset
data = open('QA_NLP/passage.txt','r',encoding="utf-8").read()

def preprocessing(rawReadCorpus):
    pattern = "^a-zA-Z0-9_"
    rawReadCorpus = contractions.fix(rawReadCorpus)
    # rawReadCorpus = pattern.sub('', str)
    # rawReadCorpus = rawReadCorpus.replace("’","qqq") # this is to make sure words like wouldn't are in a same token, after tokenisation @ would be replace with ’
    # rawReadCorpus = rawReadCorpus.replace("—"," ") # — is replaced with space
    # double quotes are removed as they prevented the sent_tokenize to separate sentences which ended with ” and are replaced with single " 
    # rawReadCorpus = rawReadCorpus.replace("“"," ")
    # rawReadCorpus = rawReadCorpus.replace("”"," ")

    text_sent = sent_tokenize(rawReadCorpus) # to split the sentences
    text_sent = [sent.lower() for sent in text_sent] # to convert to lowercase
    text_sent = ["".join([char for char in text if char not in string.punctuation ]) for text in text_sent] # removed punctuation
    text_sent = [word_tokenize(sent) for sent in text_sent]
    # text_sent = ["".join([char for char in text if char not in ]) for text in text_sent] # removed punctuation
    for i in range(len(text_sent)):
        sent = " ".join(text_sent[i])
        sent=re.sub(pattern,' ',sent)
        sent=sent.replace("“"," ")
        sent=sent.replace("”"," ")
        sent=sent.replace("—"," ")
        sent=sent.replace("_"," ")
        text_sent[i]=sent.split(' ')
        text_sent[i]=[i for i in text_sent[i] if i !='']
        # print(text_sent[i])
    i=len(text_sent)-1
    while(i>=0):
        if 'chapter' in text_sent[i]:
            print(text_sent[i])
            del(text_sent[i])
        i-=1
    return text_sent
text_sent = preprocessing(data)

['chapter', 'iv']
['chapter', 'iii']
['chapter', 'ii']
['chapter', 'i']


In [7]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 58.9 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 48.0 MB/s 
     |████████████████████████████████| 596 kB 39.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
import torch
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

In [9]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [10]:
sample = text_sent[3:6]
example=''
for sent in sample:
    example+=' '.join(sent)+'. '
    print(' '.join(sent))
# print(example)

i shall be late when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it all seemed quite natural but when the rabbit actually took a watch out of its waistcoatpocket and looked at it and then hurried on alice started to her feet for it flashed across her mind that she had never before seen a rabbit with either a waistcoatpocket or a watch to take out of it and burning with curiosity she ran across the field after it and fortunately was just in time to see it pop down a large rabbithole under the hedge
in another moment down went alice after it never once considering how in the world she was to get out again
the rabbithole went straight on like a tunnel for some way and then dipped suddenly down so suddenly that alice had not a moment to think about stopping herself before she found herself falling down a very deep well


In [11]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    # print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                    token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                    return_dict=True) 

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    # print('Answer: "' + answer + '"')
    
    return (answer,start_scores[0][answer_start], end_scores[0][answer_end])

In [12]:
# answer_question("who shall be late?",example)
from tqdm import tqdm

In [13]:
def query(question,n_sent=3):
    sent_count = len(text_sent)
    ans = 0
    ans_text = 'NA'
    for i in tqdm(range(sent_count-n_sent+1)):
        segment = text_sent[i:i+n_sent]
        passage = ''
        for sent in segment:
            passage+=' '.join(sent)+'. '
        output = answer_question(question,passage)
        if 2*(output[1]*output[2])/(output[1]+output[2]) >= ans:
            ans =  2*(output[1]*output[2])/(output[1]+output[2])
            ans_text = output[0]
    return ans_text

In [14]:
print(query("what is the colour of eyes of the white rabbit?"))

  1%|          | 3/290 [00:08<13:02,  2.73s/it]


KeyboardInterrupt: ignored

In [15]:
questions = open('QA_NLP/question_list.txt','r').readlines()

In [17]:
print(questions)

['what is the colour of eyes of the white rabbit?\n', 'What is the label on the jar in the shelf?\n', 'What was on the solid glass three legged table?\n', 'Which animal did alice speak to in french?\n', 'what are the other animals in the pool besides mouse?\n', 'What did Alice pull out of her pocket as prizes?\n', 'What was written on the door of bright brass plate?\n', 'Whom did Alice gave one sharp kick?\n', 'What happened when Alice swallowed one of the cakes?\n', 'What was the large blue caterpillar doing?\n']


In [ ]:
answer = []
for i in range(len(questions)):
    # question in questions:
    questions[i]=questions[i].replace('\n','')
    print(questions[i])
    ans = query(questions[i])
    print(ans)

what is the colour of eyes of the white rabbit?


 20%|██        | 58/290 [01:41<09:27,  2.45s/it]

# This part is for CoQA with BERT portion

In [ ]:
!git clone https://github.com/NTU-SQUAD/transformers-coqa.git

Cloning into 'transformers-coqa'...
remote: Enumerating objects: 335, done.
remote: Total 335 (delta 0), reused 0 (delta 0), pack-reused 335
Receiving objects: 100% (335/335), 90.65 KiB | 388.00 KiB/s, done.
Resolving deltas: 100% (200/200), done.


In [ ]:
# downloading coqa dataset
%cd transformers-coqa/data
!wget 'http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json'
!wget 'http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-dev-v1.0.json'
%cd ..

/content/transformers-coqa/data
--2021-10-28 05:42:47--  http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49001836 (47M) [application/json]
Saving to: ‘coqa-train-v1.0.json’

coqa-train-v1.0.jso 100%[===================>]  46.73M  8.04MB/s    in 6.6s    

2021-10-28 05:42:54 (7.12 MB/s) - ‘coqa-train-v1.0.json’ saved [49001836/49001836]

--2021-10-28 05:42:54--  http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-dev-v1.0.json
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9090845 (8.7M) [application/json]
Saving to: ‘coqa-dev-v1.0.json’

coqa-dev-v1.0.j

In [ ]:
!pip install transformers spacy tensorboard tensorboardX

     |████████████████████████████████| 2.9 MB 13.5 MB/s 
     |████████████████████████████████| 124 kB 18.3 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 45.6 MB/s 
     |████████████████████████████████| 895 kB 43.5 MB/s 
     |████████████████████████████████| 3.3 MB 39.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!python3 run_coqa.py --model_type bert \
               --model_name_or_path bert-base-uncased \
               --do_train \
               --do_eval \
               --data_dir data/ \
               --train_file coqa-train-v1.0.json \
               --predict_file coqa-dev-v1.0.json \
               --learning_rate 3e-5 \
               --num_train_epochs 2 \
               --output_dir bert-output/ \
               --do_lower_case \
               --per_gpu_train_batch_size 8  \
               --max_grad_norm -1 \
               --weight_decay 0.01 \
               --adversarial \
               --threads 8


10/28/2021 05:41:29 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
10/28/2021 05:41:30 - DEBUG - filelock -   Attempting to acquire lock 140282094898064 on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
10/28/2021 05:41:30 - DEBUG - filelock -   Lock 140282094898064 acquired on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
10/28/2021 05:41:30 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpoir8rb81
Downloading: 100% 433/433 [00:00<00:00, 346kB/s]
10/28/2021 05:41:30 - INFO - transformers.file_utils -   storing 